In [10]:
%%writefile testmapkoreng.py

import json
import streamlit as st
import pandas as pd
import folium
from streamlit_folium import folium_static
import requests
from datetime import date, datetime
import plotly.express as px
# 데이터 로딩
data = pd.read_excel("stream_data.xlsx")

# 스트림릿 상태 초기화
if 'start_date' not in st.session_state:
    st.session_state.start_date = datetime(2023, 10, 1)
if 'end_date' not in st.session_state:
    st.session_state.end_date = datetime(2023, 11, 1)

# 사이드바에 필터 추가
st.sidebar.header("Filters")
start_date = st.sidebar.date_input("Start date", st.session_state.start_date, min_value=date(2023, 10, 1))
end_date = st.sidebar.date_input("End date", st.session_state.end_date, max_value=date(2023, 11, 1))

# datetime 형식으로 변환
start_date = datetime.combine(start_date, datetime.min.time())
end_date = datetime.combine(end_date, datetime.min.time())

# st.session_state 변수 업데이트
st.session_state.start_date = start_date
st.session_state.end_date = end_date

# 선택한 날짜에 따라 데이터프레임 필터링
filtered_data = data[
    (data['일자'] >= start_date) & (data['일자'] <= end_date)
]

new_genre_list = st.sidebar.multiselect('Choose topics:', data['topics'].unique(), default=['전쟁'])
# 한국 도시와 외국 도시 데이터를 나누어 저장
korean_data = data[data['외국유무'] == '한국']
foreign_data = data[data['외국유무'] == '외국']

# tabs 만들기
tab1, tab2 = st.tabs(["국내", "해외"])

# tab1 내용물 구성하기
with tab1:
    # 제목 넣기
    st.markdown("## 국내 재난/재해 상황")

    # draw_map 함수 정의
    def draw_korea_map(start, end, topic):
        global_distriction_jsonurl = 'https://raw.githubusercontent.com/southkorea/southkorea-maps/master/kostat/2018/json/skorea-provinces-2018-geo.json'
        global_dict = json.loads(requests.get(global_distriction_jsonurl).text)

        return_df = data[data["topics"].isin(topic)]
        return_df = return_df.loc[return_df.일자.between(start, end), :]
        return_df.reset_index(drop=True, inplace=True)
        return_df = return_df.groupby(["city", "외국유무"], as_index=False)[["risk"]].sum()

        m1 = folium.Map(location=[36, 128], tiles='cartodbpositron', zoom_start=7, scrollWheelZoom=False)
        korea_result = return_df.loc[return_df["외국유무"] == "한국", :]
        choro1 = folium.Choropleth(
            data=korea_result,
            geo_data=global_dict,
            columns=["city", "risk"],
            key_on="feature.properties.name",
            mapbox_style="carto-positron",
            fill_color="YlOrRd"
        )

        choro1.add_to(m1)
        st_data = folium_static(m1, width=1000)
    # 사용자 입력을 기반으로 지도 그리기
    draw_korea_map(start_date, end_date, new_genre_list)
    
    
    
    
    col1, col2 = st.columns([5, 5])
    
    with col1:
        topics_distribution = korean_data['topics'].value_counts().reset_index()
        fig_topics = px.pie(
            topics_distribution,
            names='index',
            values='topics',
            title='한국 Topics 분포',
            hole=0.4,  # 도넛 차트 설정
            color_discrete_sequence=px.colors.qualitative.Set3  # 색상 조정
        )
        fig_topics.update_layout(
            width=400,
            height=400,
            title_x=0.5,
            margin=dict(l=0, r=0, t=30, b=0),
            showlegend=True,  # 범례 표시
            legend_title="Topics" 
        )
        st.plotly_chart(fig_topics)

# 가운데 열에는 한국 도시별 리스크 그래프 표시
    with col2:
        korean_top_5 = korean_data.groupby('city')['risk'].sum().nlargest(5).reset_index()
        fig_korean = px.bar(
            korean_top_5,
            y='risk',
            x='city',
            orientation='v',
            title="Top 5 한국 도시별 리스크",
            color='risk',  # 색상을 리스크에 따라 설정
            color_continuous_scale='viridis'  # 색상 맵 설정
        )
        fig_korean.update_layout(
            width=400,
            height=400,
            title_x=0.5,
            margin=dict(l=0, r=0, t=30, b=0),
            showlegend=True
        )
        st.plotly_chart(fig_korean)
        
        
    col1, col2 = st.columns([70, 30])
    with col1:
        st.dataframe(filtered_data[filtered_data['외국유무'] == '한국'][['일자', '제목', '요약']])

    # 상위 국가 선택을 위한 라디오 버튼
    selected_country = st.sidebar.radio("Select a country", korean_top_5['city'].head(2))
    st.write(f"Selected country: {selected_country}")
    st.write(korean_data[korean_data['city'] == selected_country])
    
# tab2 내용물 구성하기
with tab2:
    # 제목 넣기
    st.markdown("## 해외 재난/재해 상황")

    # draw_map 함수 정의
    def draw_foreign_map(start, end, topic):
        with open('국가별위도경도.json', 'r') as json_file:
            global_dict = json.load(json_file)

        return_df = data[data["topics"].isin(topic)]
        return_df = return_df.loc[return_df.일자.between(start, end), :]
        return_df.reset_index(drop=True, inplace=True)
        return_df = return_df.groupby(["city", "외국유무"], as_index=False)[["risk"]].sum()

        m2 = folium.Map(location=[31.768319, 35.21371], tiles='cartodbpositron', zoom_start=7, scrollWheelZoom=True)
        nation_result = return_df.loc[return_df["외국유무"] == "외국", :]
        choro2 = folium.Choropleth(
            data=nation_result,
            geo_data=global_dict,
            columns=["city", "risk"],
            key_on="feature.properties.name",
            mapbox_style="carto-positron",
            fill_color="YlOrRd"
        ).add_to(m2)

        choro2.add_to(m2)
        st_data = folium_static(m2, width=1000)



# 다음에 오류 수정해서 확인하면됨
    draw_foreign_map(start_date, end_date, new_genre_list)
    


        # 컬럼 설정
    col1, col2 = st.columns([5, 5])
    
    with col1:
        topics_distribution = foreign_data['topics'].value_counts().reset_index()
        fig_topics = px.pie(
            topics_distribution,
            names='index',
            values='topics',
            title='외국 Topics 분포',
            hole=0.4,  # 도넛 차트 설정
            color_discrete_sequence=px.colors.qualitative.Set3  # 색상 조정
        )
        fig_topics.update_layout(
            width=400,
            height=400,
            title_x=0.5,
            margin=dict(l=0, r=0, t=30, b=0),
            showlegend=True,  # 범례 표시
            legend_title="Topics" 
        )
        st.plotly_chart(fig_topics)
    
    with col2:
        foreign_top_5 = foreign_data.groupby('city')['risk'].sum().nlargest(5).reset_index()
        fig_foreign = px.bar(
            foreign_top_5,
            y='risk',
            x='city',
            orientation='v',
            title="Top 5 외국 도시별 리스크",
            color='risk',
            color_continuous_scale='viridis'
        )
        fig_foreign.update_layout(
            width=400,
            height=400,
            title_x=0.5,
            margin=dict(l=0, r=0, t=30, b=0),
            showlegend=True
        )
        st.plotly_chart(fig_foreign)

    
    col1, col2 = st.columns([70, 30])
    with col1:
        st.dataframe(filtered_data[filtered_data['외국유무'] == '외국'][['일자', '제목', '요약']])


Overwriting testmapkoreng.py
